<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import requests
from requests.adapters import HTTPAdapter
import pandas
import time
import tools

In [ ]:
# initialization

sess = requests.session()
sess.mount('https://', HTTPAdapter(max_retries=3))

In [ ]:

# Parameter Settings
base_url = 'https://www.airbnb.com/s/London/homes?selected_tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&screen_size=large&display_currency=USD&tab_id=home_tab&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=march&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&search_type=pagination&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=4951ed3b-6ee1-4d98-9d9c-5b5a959017a8&map_toggle=false'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0'}


In [ ]:

# Counted values
flag = 0  # 0:Next Page ；1:stop; 2:false


In [ ]:

page = 1  # log current page
data = pandas.DataFrame()  # save the data


In [ ]:

# main
while flag == 0:
    if page != 1:
        url = base_url + f'&items_offset={(page-1)*20}&section_offset=3'
    else:
        url = base_url
    rs = sess.get(url=url, headers=headers)

    # scrap the content（title, label, mark, price, detail）
    data = tools.info_ext(rs.text, data)
    if not isinstance(data, pandas.DataFrame):
        flag = 2
        break

    # net page
    print(f'page：{page}')
    print(rs.url)
    flag = page == 15
    page += 1

    # sleep
    time.sleep(1)


In [ ]:

# storage
if flag != 2:
    data.columns = ['title', 'label', 'mark', 'price', 'detail']
    data.to_excel(f'{int(time.time())}.xlsx')
